In [1]:
from config import *

from easyric.caas_lite import TiffSpliter
from easyric.objects import Pix4D
from easyric.io import shp, geotiff, plot
from easyric.calculate import geo2raw, geo2tiff

import os
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import tifffile



In [2]:
import re

# Files already have

1. non-destructive field measure
    1. 0515, 0522, 0526, 0528 -> pure field max length
    2. `ground.data_non.destructive.xlsx` -> id map + 0528 data
    
2. destructive field measure -> id map + 0528 data -> field min|max length + weight

In [9]:
root = r"Y:\hwang_Pro\data\2020_tanashi_broccoli\03_field_measurement"

In [11]:
sheet_name = ["22-May", "26-May", "28-May"]
dates = [20200522, 20200526, 20200528]

In [22]:
non_dest = pd.DataFrame(columns=["date", "Ground_ID", "UAV_ID", "flower_size_max"])

for sheet, date in zip(sheet_name, dates):
    non_link = pd.read_excel(f"{root}/ground.data_non.destructive.xlsx", sheet_name=sheet)
    non_link["date"] = date
    non_link = non_link.loc[:, ["date", "Ground_ID", "UAV_ID", "flower_size_max"]]
    non_dest = pd.concat([non_dest, non_link])
    
non_dest

,date,Ground_ID,UAV_ID,flower_size_max
0,20200522,S20_01,2916,77
1,20200522,S20_02,2915,153
2,20200522,S20_03,2914,124
3,20200522,S20_04,2913,102
4,20200522,S20_05,2912,103
...,...,...,...,...
115,20200528,N25_26,3548,157
116,20200528,N25_27,3547,168
117,20200528,N25_28,3546,166
118,20200528,N25_29,3545,146


In [29]:
non_dest.to_csv("out/non_destructive20.csv", index=False)

In [24]:
dest = pd.DataFrame(columns=["date", "Ground_ID", "UAV_ID", "flower_size_max", "flower_size_min", "weight"])

for sheet, date in zip(sheet_name, dates):
    link = pd.read_excel(f"{root}/ground.data_destructive.xlsx", sheet_name=sheet)
    link["date"] = date
    link = link.loc[:, ["date", "Ground_ID", "UAV_ID", "flower_size_max", "flower_size_min", "weight"]]
    dest = pd.concat([dest, link])
    
dest

,date,Ground_ID,UAV_ID,flower_size_max,flower_size_min,weight
0,20200522,S30_01,4368,0,0,0
1,20200522,S30_02,4367,98,81,160
2,20200522,S30_03,4366,123,105,280
3,20200522,S30_04,4365,108,96,205
4,20200522,S30_05,4364,102,92,175
...,...,...,...,...,...,...
140,20200528,S41_68,5890.0,161,133,390.0
141,20200528,S41_69,5889.0,155,145,365.0
142,20200528,S41_70,5888.0,191,165,560.0
143,20200528,S41_71,5887.0,167,164,570.0


In [26]:
dest = dest.dropna()
dest

,date,Ground_ID,UAV_ID,flower_size_max,flower_size_min,weight
0,20200522,S30_01,4368,0,0,0
1,20200522,S30_02,4367,98,81,160
2,20200522,S30_03,4366,123,105,280
3,20200522,S30_04,4365,108,96,205
4,20200522,S30_05,4364,102,92,175
...,...,...,...,...,...,...
139,20200528,S41_67,5891.0,148,125,370.0
140,20200528,S41_68,5890.0,161,133,390.0
141,20200528,S41_69,5889.0,155,145,365.0
142,20200528,S41_70,5888.0,191,165,560.0


In [27]:
dest.to_csv("out/destructive20.csv", index=False)